In [1]:
import pandas as pd
import numpy as np



In [11]:
df = pd.read_json('airbnb-listings.json')

In [21]:
x =df['fields']
rows = len(x)
facets = ['id',
 'latitude',
 'longitude',
 'name',
 'host_id',
 'host_name',
 'host_response_time',
 'host_response_rate',
 'city',
 'country',
 'property_type',
 'room_type',
 'bed_type',
 'neighbourhood_cleansed',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'extra_people',
 'price',
 'cleaning_fee',
 'cancellation_policy',
 'minimum_nights',
 'maximum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'review_scores_value',
 'review_scores_cleanliness',
 'review_scores_accuracy',
 'calculated_host_listings_count',
 'amenities',
 'features',
 'host_url']

In [22]:
x_records = []
for row in range(0,rows):
      data = x[row]
      x_records.append(data)

for ele in x_records:
    for f in facets:
        if f not in ele.keys():
            ele[f] = None

df2 = pd.DataFrame(x_records,columns=facets)
df2

In [25]:
nan_list = df2.isna().sum().to_numpy()
nan_idx = []
for i in range(len(nan_list)):
  if nan_list[i] > 0:
    tmp = []
    tmp.append(df2.keys()[i])
    tmp.append(df2[df2.keys()[i]].dtypes)
    tmp.append(nan_list[i])
    tmp.append(round(nan_list[i]/df2.shape[0] *100,2))
    nan_idx.append(tmp)
df3 = pd.DataFrame(nan_idx, columns = ["col_name","type","nan_amount","%"])
df3

,col_name,type,nan_amount,%
0,latitude,object,1,0.00
1,longitude,object,1,0.00
2,name,object,418,0.08
3,host_name,object,504,0.10
4,host_response_time,object,115069,23.25
5,host_response_rate,float64,115070,23.25
6,city,object,453,0.09
7,country,object,3,0.00
8,property_type,object,11,0.00
9,room_type,object,1,0.00


## Data Preprosessing

### Hiệu

In [ ]:
## Hiệu

### Quí

In [ ]:
## Quí

### Trí

In [54]:
df2[['review_scores_cleanliness','review_scores_value','review_scores_accuracy','reviews_per_month']][\
df2['review_scores_cleanliness'].isnull() & \
df2['review_scores_value'].isnull() & \
df2['review_scores_accuracy'].isnull() & \
df2['reviews_per_month'].isnull()]

,review_scores_cleanliness,review_scores_value,review_scores_accuracy,reviews_per_month
12,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN
34,NaN,NaN,NaN,NaN
...,...,...,...,...
494933,NaN,NaN,NaN,NaN
494934,NaN,NaN,NaN,NaN
494946,NaN,NaN,NaN,NaN
494947,NaN,NaN,NaN,NaN
